In [2]:
# ============================================================================
# MEMORY EXPERIMENT SMOKE TEST - ALL CODE TYPES
# ============================================================================
# Tests ALL codes with LER, LER-no-decode, NDR in a single unified table

import sys
import numpy as np
import warnings
from IPython.display import clear_output

# Clear any previous output
clear_output(wait=True)
warnings.filterwarnings('ignore')

# Force reload modules
for mod in list(sys.modules.keys()):
    if 'qectostim' in mod:
        del sys.modules[mod]

from qectostim.codes import discover_all_codes
from qectostim.experiments.memory import CSSMemoryExperiment, StabilizerMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise
from qectostim.decoders.decoder_selector import select_decoder

# ============================================================================
# DISCOVER AND CATEGORIZE ALL CODES
# ============================================================================
discovered = discover_all_codes(
    max_qubits=50, 
    include_qldpc=True,
    include_subsystem=True,
    include_floquet=True
)

# Categorize codes by type
all_codes = {}  # name -> (type, code)

for name, code in discovered.items():
    if 'Floquet' in name or 'Honeycomb' in name or 'ISG' in name:
        all_codes[name] = ('Floquet', code)
    elif 'Subsystem' in name or 'Gauge' in name or 'Bacon' in name:
        all_codes[name] = ('Subsystem', code)
    elif any(x in name for x in ['HGP', 'BB', 'GB', 'Hypergraph', 'Bicycle', 'Lifted', 'Fiber']):
        all_codes[name] = ('QLDPC', code)
    elif hasattr(code, 'hx') and hasattr(code, 'hz'):
        all_codes[name] = ('CSS', code)
    elif hasattr(code, 'stabilizer_matrix'):
        all_codes[name] = ('Non-CSS', code)

print(f"Discovered {len(discovered)} codes")

# ============================================================================
# TEST FUNCTION
# ============================================================================
def run_test(code, code_type, p=0.01, shots=3000, rounds=3):
    """Run memory test returning LER, LER-no-decode, NDR."""
    result = {'ler': None, 'ler_no_decode': None, 'ndr': None, 'error': None}
    
    try:
        if code_type == 'CSS' and not np.all(np.dot(code.hx, code.hz.T) % 2 == 0):
            result['error'] = 'CSS non-orthogonal'
            return result
        
        noise = CircuitDepolarizingNoise(p1=p, p2=p)
        
        if code_type == 'CSS':
            exp = CSSMemoryExperiment(code=code, rounds=rounds, noise_model=noise)
        else:
            exp = StabilizerMemoryExperiment(code=code, rounds=rounds, noise_model=noise)
        
        circuit = noise.apply(exp.to_stim())
        try:
            dem = circuit.detector_error_model(decompose_errors=True)
        except:
            try:
                dem = circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)
            except Exception as e:
                result['error'] = f'DEM: {str(e)[:25]}'
                return result
        
        sampler = dem.compile_sampler()
        raw = sampler.sample(shots, bit_packed=False)
        
        if isinstance(raw, tuple):
            det_samples = np.asarray(raw[0], dtype=np.uint8)
            obs_samples = np.asarray(raw[1], dtype=np.uint8)
        else:
            arr = np.asarray(raw, dtype=np.uint8)
            det_samples = arr[:, :dem.num_detectors]
            obs_samples = arr[:, dem.num_detectors:]
        
        if obs_samples.shape[1] > 0:
            result['ler_no_decode'] = float(obs_samples[:, 0].mean())
        
        syndrome_zero = (det_samples.sum(axis=1) == 0)
        if obs_samples.shape[1] > 0:
            obs_flipped = obs_samples[:, 0].astype(bool)
            result['ndr'] = float((syndrome_zero & obs_flipped).sum()) / shots
        
        decoder = select_decoder(dem)
        corrections = decoder.decode_batch(det_samples)
        corrections = np.asarray(corrections, dtype=np.uint8)
        if corrections.ndim == 1:
            corrections = corrections.reshape(-1, max(1, dem.num_observables))
        
        if obs_samples.shape[1] > 0:
            logical_errors = (corrections[:, 0] ^ obs_samples[:, 0]).astype(np.uint8)
            result['ler'] = float(logical_errors.mean())
        
    except Exception as e:
        result['error'] = str(e)[:30]
    
    return result

# ============================================================================
# RUN TESTS AND BUILD RESULTS
# ============================================================================
results = []
summary = {'CSS': [0,0,0], 'Non-CSS': [0,0,0], 'Subsystem': [0,0,0], 'Floquet': [0,0,0], 'QLDPC': [0,0,0]}

sorted_codes = sorted(all_codes.items(), key=lambda x: (x[1][0], x[0]))

for code_name, (code_type, code) in sorted_codes:
    n = code.n
    k = code.k
    d = code.metadata.get('distance', '?')
    
    result = run_test(code, code_type)
    
    ler_str = f"{result['ler']:.6f}" if result['ler'] is not None else "N/A"
    ler_nd_str = f"{result['ler_no_decode']:.6f}" if result['ler_no_decode'] is not None else "N/A"
    ndr_str = f"{result['ndr']:.6f}" if result['ndr'] is not None else "N/A"
    
    if result['error']:
        status = f"✗ {result['error']}"
        summary[code_type][2] += 1
    elif result['ler'] is not None:
        d_int = d if isinstance(d, int) else 0
        if result['ler'] < 1e-6 and d_int >= 3:
            status = "⚠ LER≈0"
            summary[code_type][1] += 1
        elif result['ler_no_decode'] and result['ler'] >= result['ler_no_decode'] and d_int >= 3:
            status = "⚠ Decode not helping"
            summary[code_type][1] += 1
        else:
            status = "✓ OK"
            summary[code_type][0] += 1
    else:
        status = "✗ No LER"
        summary[code_type][2] += 1
    
    results.append({
        'name': code_name, 'type': code_type, 'n': n, 'k': k, 'd': d,
        'ler': ler_str, 'ler_nd': ler_nd_str, 'ndr': ndr_str, 'status': status
    })

# ============================================================================
# DISPLAY OUTPUT (SINGLE PRINT)
# ============================================================================
output_lines = []
output_lines.append("="*140)
output_lines.append("MEMORY EXPERIMENT SMOKE TEST - ALL CODE TYPES")
output_lines.append("="*140)
output_lines.append(f"\nTesting {len(all_codes)} codes | p=0.01, shots=3000, rounds=3\n")
output_lines.append(f"{'Code':<35} | {'Type':<10} | {'n':>4} | {'k':>3} | {'d':>3} | {'LER':>10} | {'LER-no-dec':>10} | {'NDR':>10} | {'Status':<30}")
output_lines.append("-"*140)

for r in results:
    output_lines.append(f"{r['name']:<35} | {r['type']:<10} | {r['n']:>4} | {r['k']:>3} | {str(r['d']):>3} | {r['ler']:>10} | {r['ler_nd']:>10} | {r['ndr']:>10} | {r['status']:<30}")

output_lines.append("-"*140)
output_lines.append("")
output_lines.append("="*60)
output_lines.append("SUMMARY")
output_lines.append("="*60)
output_lines.append(f"\n{'Type':<12} {'Total':>8} {'Pass':>8} {'Warn':>8} {'Fail':>8}")
output_lines.append("-"*48)

total = [0, 0, 0]
for code_type in ['CSS', 'Non-CSS', 'Subsystem', 'Floquet', 'QLDPC']:
    p, w, f = summary[code_type]
    t = p + w + f
    if t > 0:
        output_lines.append(f"{code_type:<12} {t:>8} {p:>8} {w:>8} {f:>8}")
        total[0] += p
        total[1] += w
        total[2] += f

output_lines.append("-"*48)
output_lines.append(f"{'TOTAL':<12} {sum(total):>8} {total[0]:>8} {total[1]:>8} {total[2]:>8}")
output_lines.append("")
output_lines.append("="*60)
if total[2] == 0:
    output_lines.append("✓ ALL CODES PASS")
else:
    output_lines.append(f"⚠ {total[2]} CODES FAILED - Need fixing in codebase")
output_lines.append("="*60)

# Single print statement to avoid any output duplication
print("\n".join(output_lines))

Discovered 32 codes
MEMORY EXPERIMENT SMOKE TEST - ALL CODE TYPES

Testing 32 codes | p=0.01, shots=3000, rounds=3

Code                                | Type       |    n |   k |   d |        LER | LER-no-dec |        NDR | Status                        
--------------------------------------------------------------------------------------------------------------------------------------------
C6                                  | CSS        |    6 |   2 |   2 |   0.083667 |   0.083667 |   0.000333 | ✓ OK                          
Code_832                            | CSS        |    8 |   3 |   2 |   0.008000 |   0.079000 |   0.000000 | ✓ OK                          
FourQubit422_[[4,2,?]]              | CSS        |    4 |   2 |   2 |   0.055333 |   0.055333 |   0.000667 | ✓ OK                          
Hamming_CSS_7                       | CSS        |    7 |   1 |   3 |   0.090333 |   0.103000 |   0.000000 | ✓ OK                          
HexagonalColour_d3                  | CSS  